### DNN기반 다중분류 모델 구현
- 데이터셋  : iris.csv
- 피쳐/속성 : 4개 (Sepal_Length, Sepal_Width, Petal_Length, Petal_Width)
- 타겟/라벨 : 1개 (Variety)
- 학습 방법 : 지도학습 > 분류
- 알고리즘  : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
- 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비 <hr>

In [26]:
# 모듈 로딩
# Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.classification import MulticlassF1Score
from torchmetrics.classification import BinaryConfusionMatrix
from torchinfo import summary

# Data 및 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [27]:
# 활용 패키지 버전 체크 ==> 사용자 정의함수로 구현하세요~
print(f'Pytorch v.{torch.__version__}')
print(f'pandas v.{pd.__version__}')

Pytorch v.2.4.1
pandas v.2.0.3


In [28]:
# 데이터 로딩
DATA_FILE = '../../../Data/iris.csv'

# CSV >>> DataFrame
irisDF = pd.read_csv(DATA_FILE)

# 확인
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [29]:
# 타겟 변경 => 정수화, 클래스 3개
irisDF['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [30]:
labels = dict(zip(irisDF['variety'].unique().tolist(),range(3)))
print(f'labels => {labels}')
irisDF['variety'] = irisDF['variety'].replace(labels)
irisDF.head(1)

labels => {'Setosa': 0, 'Versicolor': 1, 'Virginica': 2}


,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0


In [31]:
print(f"고유값 : {irisDF['variety'].unique()}")

고유값 : [0 1 2]


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스 목적 : iris 데이터를 학습 및 추론 목적
- 클래스 이름 : IrisMCFModel
- 부모 클래스 : nn.Module
- 매개   변수 : 층별 입출력 개수 고정하기 때문에 필요 없음
- 속성 / 필드 :
- 기능 / 역할 : \__init__() : 모델 구조 설정, forward() : 순방향 학습 <= 오버라이딩(overriding)

- 클래스 구조
    * 입력층 :
        * 입력 4개(피쳐)
        * 출력 10개(퍼셉트론/뉴런 10개 존재)
    * 은닉층 :
        * 입력 10개
        * 출력 5개(퍼셉트론/뉴런 30개 존재)
    * 출력층 :
        * 입력 5개
        * 출력 3개(퍼셉트론/뉴런 1개 존재 : 2진분류)

- 손실함수/활성화함수
    * 클래스 형태 ==> nn.MSELoss, nn.ReLU ==> init() 메서드
    * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드

In [32]:
class IrisMCFModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(4, 10)
        self.hidden_layer = nn.Linear(10, 5)
        self.out_layer = nn.Linear(5, 3) # 다중분류 'Setosa', 'Versicolor', 'Virginica'

    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # 입력층
        y = self.in_layer(input_data)
        y = F.relu(y)

        # 은닉층 : 10개의 숫자 값(>=0)
        y = self.hidden_layer(y)
        y = F.relu(y)

        # 출력층 : 5개의 숫자 값 => 다중 분류 : 손실함수 CrossEntropyLoss가 내부에서 softmax 진행
        return self.out_layer(y)

In [33]:
# 모델 인스턴스 생성
model = IrisMCFModel()

print(model)

IrisMCFModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=3, bias=True)
)


In [34]:
# 모델 사용
summary(model, input_size=(5000000,4))

Layer (type:depth-idx)                   Output Shape              Param #
IrisMCFModel                             [5000000, 3]              --
├─Linear: 1-1                            [5000000, 10]             50
├─Linear: 1-2                            [5000000, 5]              55
├─Linear: 1-3                            [5000000, 3]              18
Total params: 123
Trainable params: 123
Non-trainable params: 0
Total mult-adds (M): 615
Input size (MB): 80.00
Forward/backward pass size (MB): 720.00
Params size (MB): 0.00
Estimated Total Size (MB): 800.00

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐 개수 : 4개
- 타겟 개수 : 3개
- 클래스 이름 : IrisDataset
- 부모 클래스 : utils.data.Dataset
- 속성 필드 : featureDF, targetDF, n_rows, n_features
- 필수 메서드
    * \__init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * \__len__(self) : 데이터의 개수 반환
    * \__getitem__(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [35]:
class IrisDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)
        
        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [36]:
# [테스트] 데이터셋 인스턴스 생성

# DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]] # 2D (150, 3)
targetDF = irisDF[irisDF.columns[-1:]] # 2D (150, 1)

# 커스텀 데이터셋 인스턴스
irisDS = IrisDataset(featureDF, targetDF)

# 데이터로더 인스턴스 생성
irisDL = DataLoader(irisDS)
for feature, label in irisDL:
    print(feature.shape, label.shape, feature, label)
    break

torch.Size([1, 4]) torch.Size([1, 1]) tensor([[5.1000, 3.5000, 1.4000, 0.2000]]) tensor([[0.]])


[4] 학습 준비
- 학습 횟수 : EPOCH (처음부터 끝까지 학습하는 단위)
- 배치 크기 : BATCH_SIZE (한 번에 학습할 데이터셋 양)
- 위치 지정 : DEVICE (텐서 저장 및 실행 위치 : GPU/CPU) 
- 학습률 : LR 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001 ~ 0.1

In [37]:
# 학습 진행 관련 설정
EPOCH = 1000
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

- 인스턴스/객체 : 모델, 데이터셋, 최적화 (, 손실함수, 성능지표)

In [38]:
# 모델 인스턴스
model = IrisMCFModel()

# DS와 DL 인스턴스
# 학습용, 검증용, 테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{type(y_train)} {type(y_test)} {type(y_val)}')

# 학습용, 검증용, 테스트용 데이터셋
trainDS = IrisDataset(X_train, y_train)
valDS  = IrisDataset(X_val, y_val)
testDS = IrisDataset(X_test, y_test)

# 학습용 데이터로더 인스턴스
trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE)

(84, 4) (38, 4) (28, 4)
(84, 1) (38, 1) (28, 1)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [39]:
# [테스트] 데이터로더
for feature, target in trainDL:
    print(feature.shape, target.shape, sep='\n')
    break

torch.Size([10, 4])
torch.Size([10, 1])


- 최적화 손실함수 인스턴스

In [40]:
# 최적화 인스턴스 => W, b 텐서 즉, model.parameters() 전달
optimizer = optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 분류 => 다중분류 CrossEntropyLoss
#                              예측값은 선형식 결과값 전달 ==> AF 처리 X
crossLoss = nn.CrossEntropyLoss()

[5] 학습 진행

In [41]:
len(trainDL), trainDL.__len__()

(9, 9)

In [42]:
# 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]], [[],[]]
CNT = len(trainDL)
print(f"BATCH_CNT => {CNT}")

for epoch in range(EPOCH):
    # 학습 모드로 모델 설정
    model.train()

    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0, 0

    for featureTS, targetTS in trainDL:
        # 학습 진행
        pred_y = model(featureTS)

        # 손실 계산 : nn.CrossEntropyLoss 요구사항 : 정답/타겟은 0D 또는 1D, 타입은 long
        loss = crossLoss(pred_y, targetTS.reshape(-1).long())
        loss_total += loss.item()

        # 성능평가 계산
        score = MulticlassF1Score(num_classes=3)(pred_y, targetTS.reshape(-1).long())
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()

    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)

        # 추론/평가
        pred_test_y = model(val_featureTS)
        # 손실 계산
        loss_val = crossLoss(pred_test_y, val_targetTS.reshape(-1).long())
        # 성능 평가
        score_val = MulticlassF1Score(num_classes=3)(pred_test_y, val_targetTS.reshape(-1).long())

    

    # 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/len(trainDL))
    SCORE_HISTORY[0].append(score_total/len(trainDL))

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    print(f'[{epoch}/{EPOCH}]\n- [TRAIN] LOSS : {LOSS_HISTORY[0][-1]} SCORE : {SCORE_HISTORY[0][-1]}')
    print(f'- [VALID] LOSS : {LOSS_HISTORY[1][-1]} SCORE : {SCORE_HISTORY[1][-1]}')

BATCH_CNT => 9
[0/1000]
- [TRAIN] LOSS : 1.2174387905332777 SCORE : 0.16442138453324637
- [VALID] LOSS : 1.3402509689331055 SCORE : 0.10101010650396347
[1/1000]
- [TRAIN] LOSS : 1.1987730198436313 SCORE : 0.16442138453324637
- [VALID] LOSS : 1.3168338537216187 SCORE : 0.10101010650396347
[2/1000]
- [TRAIN] LOSS : 1.1830169293615553 SCORE : 0.16442138453324637
- [VALID] LOSS : 1.2963553667068481 SCORE : 0.10101010650396347
[3/1000]
- [TRAIN] LOSS : 1.1701056626107957 SCORE : 0.16442138453324637
- [VALID] LOSS : 1.2792949676513672 SCORE : 0.10101010650396347
[4/1000]
- [TRAIN] LOSS : 1.1593498190244038 SCORE : 0.16442138453324637
- [VALID] LOSS : 1.2629916667938232 SCORE : 0.10101010650396347
[5/1000]
- [TRAIN] LOSS : 1.1468726793924968 SCORE : 0.16442138453324637
- [VALID] LOSS : 1.2400211095809937 SCORE : 0.10101010650396347
[6/1000]
- [TRAIN] LOSS : 1.1299846569697063 SCORE : 0.16442138453324637
- [VALID] LOSS : 1.213242530822754 SCORE : 0.10101010650396347
[7/1000]
- [TRAIN] LOSS : 1

- 학습결과 체크 => 학습과 검증의 Loss 변화, 성능 변화

In [46]:
# 모델 테스트 모드 설정
model.eval()

with torch.no_grad():
    # 검증 데이터셋
    test_featureTS = torch.FloatTensor(testDS.featureDF.values)
    test_targetTS = torch.FloatTensor(testDS.targetDF.values)

    # 추론/평가
    pred_test_y = model(test_featureTS)
    print(pred_test_y)

    # 손실 계산
    loss_test = crossLoss(pred_test_y, test_targetTS.reshape(-1).long())
    # 성능 평가
    score_test = MulticlassF1Score(num_classes=3)(pred_test_y, test_targetTS.reshape(-1).long())

tensor([[  8.2828,  -8.8570,  -7.2254],
        [ -3.0037,   2.6675,  -1.5419],
        [ -3.1029,   2.7687,  -1.4919],
        [  7.2342,  -7.7864,  -6.6974],
        [-10.6571,  -0.6151,   1.8705],
        [ -3.8283,   2.4427,  -1.1675],
        [-14.1606,  -2.1455,   3.3761],
        [  5.8932,  -6.4171,  -6.0221],
        [  5.6090,  -6.1269,  -5.8790],
        [-16.3660,  -3.1329,   4.3571],
        [ -3.1029,   2.7687,  -1.4919],
        [  6.8803,  -7.4250,  -6.5192],
        [-15.9491,  -2.9540,   4.1824],
        [ -3.1029,   2.7687,  -1.4919],
        [ -4.8006,   2.0135,  -0.7435],
        [  5.7695,  -6.2908,  -5.9598],
        [ -3.1029,   2.7687,  -1.4919],
        [ -4.8222,   2.0059,  -0.7367],
        [  6.1017,  -6.6300,  -6.1271],
        [  6.4538,  -6.9894,  -6.3044],
        [ -3.6099,   2.5421,  -1.2668],
        [ -5.1561,   1.8585,  -0.5911],
        [ -7.6878,   0.7238,   0.5366],
        [  6.5369,  -7.0744,  -6.3463],
        [-11.4761,  -0.9730,   2.2227],


In [47]:
score_test

tensor(1.)